# Project MTI865 - Heart segmentation using UNet 

---

# Model evaluation 

## Importing libraries 

In [1]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import v2
from progressBar import printProgressBar

import medicalDataLoader
import argparse
import utils

from UNet_Base import *
import random
import torch
import pdb
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn import metrics as skmetrics
from scipy import stats 
import metrics 

import warnings
warnings.filterwarnings("ignore") 
import pandas as pd 


## Loading data 

### Batch sizes

In [2]:
batch_size = 4
batch_size_val = 4
batch_size_unlabel = 4

### Mask and image transformation

In [3]:
# Define image and mask transformations
transform = v2.Compose([
    v2.ToTensor()
])

mask_transform = v2.Compose([
    v2.ToTensor()
])

### Data loaders 

In [4]:
def collate_fn(batch):
    imgs = []
    masks = []
    img_paths = []

    for item in batch:
        img, mask, img_path = item[0], item[1], item[2]
        imgs.append(img)
        img_paths.append(img_path)
        
        # Si le masque est None, ajouter un tenseur de zéros correspondant à sa taille
        if mask is not None:
            masks.append(mask)
        else:
            masks.append(torch.zeros_like(img[0, :, :]))  # Même taille que le canal de l'image (assumant CxHxW)

    # Stack les images et les masques
    imgs_tensor = torch.stack(imgs)  # Tensor de forme (B, C, H, W)
    masks_tensor = torch.stack(masks)  # Tensor de forme (B, H, W)

    return imgs_tensor, masks_tensor, img_paths

In [5]:
# Define dataloaders
root_dir = './data/'
print(' Dataset: {} '.format(root_dir))

supervised_set = medicalDataLoader.MedicalImageDataset('train',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    augment=True,
                                                    equalize=False)


supervised_loader = DataLoader(
    supervised_set,
    batch_size=batch_size,
    worker_init_fn=np.random.seed(0),
    num_workers=0,
    shuffle=True,
    collate_fn=collate_fn)


val_set = medicalDataLoader.MedicalImageDataset('val',
                                                root_dir,
                                                transform=transform,
                                                mask_transform=mask_transform,
                                                equalize=False)

val_loader = DataLoader(val_set,
                        batch_size=batch_size_val,
                        worker_init_fn=np.random.seed(0),
                        num_workers=0,
                        shuffle=False)

unsupervised_set = medicalDataLoader.MedicalImageDataset('train-unlabelled',
                                                            root_dir,
                                                            transform=transform,
                                                            mask_transform=mask_transform,
                                                            augment=False,
                                                            equalize=False)
# print(train_unlabelled_set.imgs)
# train_unlabelled_set = [(img) for img, mask in train_unlabelled_set]
unsupervised_loader = DataLoader(unsupervised_set,
                                    batch_size=batch_size_unlabel,
                                    worker_init_fn=np.random.seed(0),
                                    num_workers=0,
                                    shuffle=False,
                                    collate_fn=collate_fn)

test_set = medicalDataLoader.MedicalImageDataset('test',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    augment=True,
                                                    equalize=False)

test_loader = DataLoader(
    test_set,
    batch_size=batch_size_unlabel,
    worker_init_fn=np.random.seed(0),
    num_workers=0,
    shuffle=False,
    collate_fn=collate_fn)


# Let's print the first batch to understand the data

for loader in [supervised_loader, val_loader, unsupervised_loader, test_loader]:
    imgs, masks, img_paths = next(iter(loader))
    print('Images shape: ', imgs.shape)
    print('Masks shape: ', masks.shape)
    # print('Image paths: ', img_paths)



 Dataset: ./data/ 
Found 204 items in train
First item:  ('./data/train\\Img\\patient006_01_1.png', './data/train\\GT\\patient006_01_1.png')
Found 74 items in val
First item:  ('./data/val\\Img\\patient001_01_1.png', './data/val\\GT\\patient001_01_1.png')
Found 1004 items in train-unlabelled
First item:  ('./data/train\\Img-Unlabeled\\patient007_01_1.png', None)
Found 314 items in test
First item:  ('./data/test\\Img\\patient002_01_1.png', './data/test\\GT\\patient002_01_1.png')
Images shape:  torch.Size([4, 1, 256, 256])
Masks shape:  torch.Size([4, 1, 256, 256])
Images shape:  torch.Size([4, 1, 256, 256])
Masks shape:  torch.Size([4, 1, 256, 256])
Images shape:  torch.Size([4, 1, 256, 256])
Masks shape:  torch.Size([4, 256, 256])
Images shape:  torch.Size([4, 1, 256, 256])
Masks shape:  torch.Size([4, 1, 256, 256])


## Loading a model 

### Loading the parameters

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.mps.is_available():  # Apple M-series of chips
#     device = torch.device("mps")
else:
    device = torch.device("cpu")

epoch_to_load = 93
model = UNet(4).to(device=device)
modelName = 'Test_Model'
# model.load_state_dict(torch.load(f"./models/{modelName}/{epoch_to_load}_Epoch"))

model.load_state_dict(torch.load("save/DSC-CE/61_Epoch-0.3DSC+0.7CE"))


<All keys matched successfully>

In [72]:

dice_score_array = metrics.Dice_score_class(model, test_loader, device)
iou_score_array = metrics.Jaccard_score_class(model, test_loader, device)
hsd_score_array = metrics.hsd_score_class(model, test_loader, device)
precision_score_array = metrics.precision_class(model, test_loader, device)
recall_score_array = metrics.recall_class(model, test_loader, device)
f1_score_array = metrics.f1_score_class(model, test_loader, device)
auc_score_array = metrics.auc_coeff_class(model, test_loader, device)
accuracy_score_array = metrics.accuracy_class(model, test_loader, device)
          


In [73]:
print('Dice score: ', dice_score_array)
print(dice_score_array[1:])

Dice score:  [0.9937838100803682, 0.42750724548692304, 0.4680207024956805, 0.6301194251136166]
[0.42750724548692304, 0.4680207024956805, 0.6301194251136166]


In [74]:
dice_mean = np.mean(dice_score_array)
iou_mean = np.mean(iou_score_array)
hsd_mean = np.mean(hsd_score_array)
precision_mean = np.mean(precision_score_array)
recall_mean = np.mean(recall_score_array)
f1_mean = np.mean(f1_score_array)
auc_mean = np.mean(auc_score_array)
accuracy_mean = np.mean(accuracy_score_array)

dice_mean_without_bg = np.mean(dice_score_array[1:])
iou_mean_without_bg = np.mean(iou_score_array[1:])
hsd_mean_without_bg = np.mean(hsd_score_array[1:])
precision_mean_without_bg = np.mean(precision_score_array[1:])
recall_mean_without_bg = np.mean(recall_score_array[1:])
f1_mean_without_bg = np.mean(f1_score_array[1:])
auc_mean_without_bg = np.mean(auc_score_array[1:])
accuracy_mean_without_bg = np.mean(accuracy_score_array[1:])



In [75]:

# building the data frame 
data = {'Dice': dice_mean, 'IoU': iou_mean, 'HSD': hsd_mean, 'Precision': precision_mean, 'Recall': recall_mean, 'F1': f1_mean, 'AUC': auc_mean, 'Accuracy': accuracy_mean}
data_without_bg = {'Dice': dice_mean_without_bg, 'IoU': iou_mean_without_bg, 'HSD': hsd_mean_without_bg, 'Precision': precision_mean_without_bg, 'Recall': recall_mean_without_bg, 'F1': f1_mean_without_bg, 'AUC': auc_mean_without_bg, 'Accuracy': accuracy_mean_without_bg}

# Create DataFrame
df_mean = pd.DataFrame(data, index =['Mean'])
df_without_bg = pd.DataFrame(data_without_bg, index =['Mean'])


In [76]:
print(df_mean)

          Dice     IoU       HSD  Precision    Recall        F1       AUC  \
Mean  0.629858  0.5422  18.08874   0.652928  0.645326  0.620209  0.744082   

      Accuracy  
Mean  0.991937  


In [77]:
print(df_without_bg)

          Dice       IoU        HSD  Precision    Recall        F1       AUC  \
Mean  0.508549  0.393751  13.503145   0.539033  0.529358  0.495671  0.714126   

      Accuracy  
Mean    0.9933  


In [78]:
data_by_class = {'Dice': dice_score_array, 'IoU': iou_score_array, 'HSD': hsd_score_array, 'Precision': precision_score_array, 'Recall': recall_score_array, 'F1': f1_score_array, 'AUC': auc_score_array, 'Accuracy': accuracy_score_array}
index = ['Background', 'RV', 'Myo', 'LV']
df_by_class = pd.DataFrame(data_by_class, index=index)
print(df_by_class)

                Dice       IoU        HSD  Precision    Recall        F1  \
Background  0.993784  0.987547  31.845524   0.994614  0.993230  0.993820   
RV          0.427507  0.335838  40.509435   0.455080  0.463785  0.413471   
Myo         0.468021  0.329313   0.000000   0.508702  0.473089  0.459968   
LV          0.630119  0.516104   0.000000   0.653316  0.651199  0.613575   

                 AUC  Accuracy  
Background  0.833950  0.987846  
RV          0.636150  0.991724  
Myo         0.705835  0.992193  
LV          0.800395  0.995984  


In [79]:
# to csv 
df_mean.to_csv('mean_metrics.csv')
df_without_bg.to_csv('mean_metrics_without_bg.csv')
df_by_class.to_csv('metrics_by_class.csv')